# AI Agents in LangGraph – Introduction & Lesson 1

Notes from the short course on building **AI agents** with the **LangGraph** extension of LangChain. Recent progress in **function‑calling LLMs** and purpose‑built tools (e.g. agent‑aware search) has made agentic applications *predictable* and *stable* to run in production.

Successful teams follow a handful of design patterns:
1. **Planning** – break the goal into executable steps.
2. **Tool use** – route calls to well‑defined interfaces (APIs, search, calculators…).
3. **Reflection** – critique and refine intermediate results.
4. **Multi‑agent collaboration** – specialise roles (planner, researcher, writer…).
5. **Memory & persistence** – record state for long‑running tasks and debugging.

**LangGraph** lets users express cyclic graphs that capture these patterns declaratively, while still using LangChain’s rich ecosystem of tools and observability hooks.

In *Lesson 1* a minimal **ReAct** agent is build from scratch. Future lessons will re‑implement it with LangGraph and extend it with the patterns above.

## 0 · Environment setup
Install the required packages once:
```bash
pip install openai python-dotenv
```
Store your OpenAI key in a local **.env** file (never commit secrets!):
```bash
echo "OPENAI_API_KEY=sk-…" >> .env
echo ".env" >> .gitignore
```

In [1]:
from dotenv import load_dotenv
import os, re, math

load_dotenv()
assert os.getenv('OPENAI_API_KEY'), "⚠️  OPENAI_API_KEY not found. Did you create the .env file?"

# If you need the OpenAI client:
from openai import OpenAI
client = OpenAI()


# Lesson 1 — Building a ReAct Agent from Scratch

The **ReAct** pattern interleaves *reasoning* with *actions* on external tools. Formally, at turn $t$ we have state $s_t$. The policy chooses an action $a_t \sim \pi_\theta(\cdot \mid s_t)$, observes $o_t$ from the environment, and updates the state $s_{t+1}=f(s_t, a_t, o_t)$.

Everything in the cell below is handled by the LLM; our job is to write the runtime glue‑code.

In [2]:
class Agent:
    """A minimal ReAct agent using an OpenAI chat model."""
    def __init__(self, system: str = ""):
        # Initialize the agent with a system message
        self.messages = []
        # Add a system message if provided
        if system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, user_message: str) -> str:
        # Add a user message
        self.messages.append({"role": "user", "content": user_message})
        # Get the response from the OpenAI API model
        result= self.execute()
        # Add the assistant message
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o",
            temperature=0,
            messages=self.messages)
        return completion.choices[0].message.content
        


### Prompt with available tools

In [3]:
# ReAct Agent Prompt
prompt_react = '''You run in a loop of Thought, Action, PAUSE, Observation. At the end of the loop you output an Answer.

Use Thought to describe your reasoning about the question.
Use Action to run one of the tools, then return PAUSE.
The runtime will feed you the Observation on the next turn.

Available tools:
* calculate: e.g. `calculate: 4 * 7 / 3` – run a Python calculation.
* average_dog_weight: e.g. `average_dog_weight: Collie` – return the breed’s average weight.

Example session:
Question: How much does a Bulldog weigh?
Thought: I should look up the dog's weight.
Action: average_dog_weight: Bulldog
PAUSE'''.strip()

In [4]:
# ReAct Agent Tools Definitions - are the tools mentioned in the prompt above

def calculate(expr: str):
    # Use a safe parser in production; eval is for demo only.
    return eval(expr)

def average_dog_weight(breed: str):
    if breed in "Scottish Terrier":
        return("Scottish Terrier average weight is 20 lbs")
    elif breed in "Border Collie":
        return("Border Collie average weight is 37 lbs")
    elif breed in "Toy Poodle":
        return("Toy Poodle average weight is 7 lbs")
    else:
        return("An average dog weight is 50 lbs")

KNOWN_ACTIONS = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight,
}


In [5]:
# Example of calling manually each step

abot = Agent(prompt_react)
result= abot("How much does a toy poddle weigh?")
print(result)
result = average_dog_weight("Toy Poodle")
print(result)
next_prompt = "Obsservation: {}".format(result)
abot(next_prompt)
print(abot.messages)

Thought: I should look up the average weight of a Toy Poodle.
Action: average_dog_weight: Toy Poodle
PAUSE
Toy Poodle average weight is 7 lbs
[{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation. At the end of the loop you output an Answer.\n\nUse Thought to describe your reasoning about the question.\nUse Action to run one of the tools, then return PAUSE.\nThe runtime will feed you the Observation on the next turn.\n\nAvailable tools:\n* calculate: e.g. `calculate: 4 * 7 / 3` – run a Python calculation.\n* average_dog_weight: e.g. `average_dog_weight: Collie` – return the breed’s average weight.\n\nExample session:\nQuestion: How much does a Bulldog weigh?\nThought: I should look up the dog's weight.\nAction: average_dog_weight: Bulldog\nPAUSE"}, {'role': 'user', 'content': 'How much does a toy poddle weigh?'}, {'role': 'assistant', 'content': 'Thought: I should look up the average weight of a Toy Poodle.\nAction: average_dog_weight: Toy Poodle\nPAU

In [6]:
# Example loop to call the agent

# Python regular expression to select action
ACTION_RE = re.compile(r'^Action: (\w+): (.*)$')

# Function to call the agent, max_turns is the maximum number of turns
# before giving up on the agent
def query(question: str, max_turns: int = 5):
    # Create the agent
    bot = Agent(prompt_react)
    next_prompt = question
    # Loop for max_turns times
    for _ in range(max_turns):
        # Call the agent and get the result
        result = bot(next_prompt)
        print("\nLLM →\n" + result)
        # Extract the action from the result
        m = next((ACTION_RE.match(line) for line in result.split('\n') if ACTION_RE.match(line)), None)
        if not m:  # The agent answered
            break
        action, arg = m.groups()
        # Check if the action is known      
        if action not in KNOWN_ACTIONS:
            raise ValueError(f"Unknown action: {action}")
        # Call the action and get the observation
        observation = KNOWN_ACTIONS[action](arg.strip())
        print(f"[runtime] {action}({arg}) → {observation}")
        # Create the next prompt
        next_prompt = f"Observation: {observation}"


In [7]:
# ▶️ Try it!
query("I have 2 dogs, a border collie and a scottish terrier. What is their combined weight?")


LLM →
Thought: I need to find the average weight of both a Border Collie and a Scottish Terrier, then add them together to get the combined weight.
Action: average_dog_weight: Border Collie
PAUSE
[runtime] average_dog_weight(Border Collie) → Border Collie average weight is 37 lbs

LLM →
Action: average_dog_weight: Scottish Terrier
PAUSE
[runtime] average_dog_weight(Scottish Terrier) → Scottish Terrier average weight is 20 lbs

LLM →
Thought: Now that I have the average weights of both dogs, I can calculate their combined weight by adding the two values together.
Action: calculate: 37 + 20
PAUSE
[runtime] calculate(37 + 20) → 57

LLM →
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.


### Notes on Agent
* **Determinism** – `temperature=0` keeps tool calls parseable.
* **Security** – replace `eval` with `asteval` or `simpleeval` in production.
* **Observability** – `Agent.messages` lets you inspect the full trajectory.
* **Cost control** – messages carry only the working context, keeping token bills down. If blind append every turn forever, the prompt grows quadratically and costs explode. 